In [1]:
import os
import json
from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict
from marker.output import text_from_rendered
from tqdm import tqdm
import configparser





In [2]:
config = configparser.ConfigParser()

# Read the ini file
config.read("config.ini")

# Access values
metadata_path = config["paths"]["metadata"]
pdfs_path = config["paths"]["pdfs"]
markdowns_path = config["paths"]["markdowns"]


metadata_files = os.listdir(metadata_path)
metadata = []

# read json metadata files
for file in tqdm(metadata_files):
    with open(os.path.join(metadata_path, file), 'r') as f:
        metadata.append(json.load(f))
        


100%|██████████| 773/773 [00:00<00:00, 3177.29it/s]


In [3]:

converter = PdfConverter(
    
    artifact_dict=create_model_dict(),
)

Loaded layout model datalab-to/surya_layout on device cuda with dtype torch.float16
Loaded texify model to cuda with torch.float16 dtype
Loaded recognition model vikp/surya_rec2 on device cuda with dtype torch.float16
Loaded table recognition model datalab-to/surya_tablerec on device cuda with dtype torch.float16
Loaded detection model vikp/surya_det3 on device cuda with dtype torch.float16


In [15]:
metadata[0]

{'Titre': 'Absence de brises soleil et de stores',
 'Texte': "Des feuilles de papier sont collées sur le vitrage en guide de protection contre l'éblouissement.",
 'Images': ['https://www.dispositif-rexbp.com/sites/default/files/2023-11/D%C3%A9faut%20de%20protections%20solaires%20chaleur%20et%20%C3%A9blouissement.jpg'],
 'Videos': [None],
 'PDFs': ['https://www.dispositif-rexbp.com/sites/default/files/2023-11/Eblouissement%20et%20surchauffe_1.pdf'],
 'Thématique': ['Éclairage et électricité spécifique'],
 'Type_document': ['Photos commentées'],
 'Lien': 'https://www.dispositif-rexbp.com/ressource/absence-de-brises-soleil-et-de-stores'}

In [4]:
def pdf_name_to_path(pdf_name):
    return os.path.join(pdfs_path, f"{pdf_name}.pdf")

def transform_pdf_to_markdown(converter,metadata):
    
    title = metadata["Titre"]
    pdf_path = pdf_name_to_path(title)   
    output_file_path = os.path.join(markdowns_path, f"{title}.md")
    images_output_path = os.path.join(markdowns_path, "images" , title)
    if os.path.exists(output_file_path): 
        print(f"markdown {output_file_path} already exists, skipping ... ")
        return
    if not os.path.exists(pdf_path): 
        print(f"pdf {pdf_path} not found, skipping ... ")
        return
    
    print(f"converting {pdf_path} to markdown ...")
    rendered = converter(pdf_path)
    

    text, ext, images = text_from_rendered(rendered)

    #transform the refrences of the images inside the text to the new path
    for img in images:
        text = text.replace(img,f"<images/{title}/{os.path.basename(img)}>")
    
    with open(output_file_path, 'w') as f:
        f.write(text)
    
    # save the images
    if not os.path.exists(images_output_path):
        os.makedirs(images_output_path)
    for img in images:
        img_path = os.path.join(images_output_path,os.path.basename(img))
        try :
            images[img].save(img_path)
        except:
            print(f"error saving {img_path}")
            continue

        


In [5]:
for m in tqdm(metadata):
    transform_pdf_to_markdown(converter,m)

  0%|          | 0/773 [00:00<?, ?it/s]

markdown D:\upec\IA2S\PDF_SCRAPPER\MARKDOWN_DATA\Absence de brises soleil et de stores.md already exists, skipping ... 
markdown D:\upec\IA2S\PDF_SCRAPPER\MARKDOWN_DATA\Absence de développement de la végétation.md already exists, skipping ... 
markdown D:\upec\IA2S\PDF_SCRAPPER\MARKDOWN_DATA\Absence de fixation du CET monobloc.md already exists, skipping ... 
markdown D:\upec\IA2S\PDF_SCRAPPER\MARKDOWN_DATA\Absence vase expansion.md already exists, skipping ... 
markdown D:\upec\IA2S\PDF_SCRAPPER\MARKDOWN_DATA\Accès difficile voire impossible pour l'entretien..md already exists, skipping ... 
markdown D:\upec\IA2S\PDF_SCRAPPER\MARKDOWN_DATA\Agir sur la production d'eau chaude sanitaire.md already exists, skipping ... 
markdown D:\upec\IA2S\PDF_SCRAPPER\MARKDOWN_DATA\Agir sur la production d'électricité.md already exists, skipping ... 
markdown D:\upec\IA2S\PDF_SCRAPPER\MARKDOWN_DATA\Agir sur la production de chauffage-refroidissement.md already exists, skipping ... 
markdown D:\upec\IA

100%|██████████| 8/8 [00:00<00:00, 47.87it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing equations: 100%|██████████| 1/1 [00:00<00:00,  1.13it/s]
Detecting bboxes: 0it [00:00, ?it/s]
 76%|███████▌  | 585/773 [00:26<00:08, 22.49it/s]

Overflow in columns: 6 >= 6
markdown D:\upec\IA2S\PDF_SCRAPPER\MARKDOWN_DATA\Protection de chantier non étanche.md already exists, skipping ... 
markdown D:\upec\IA2S\PDF_SCRAPPER\MARKDOWN_DATA\Protection de chantier étanche.md already exists, skipping ... 
converting D:\pdf_data\Protections solaires des façades en climat tropical - 12 enseignements à connaître.pdf to markdown ...


100%|██████████| 5/5 [00:00<00:00, 54.63it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing equations: 0it [00:00, ?it/s]
Detecting bboxes: 0it [00:00, ?it/s]
 76%|███████▌  | 588/773 [00:43<00:15, 11.63it/s]

...

100%|██████████| 1/1 [00:00<00:00, 28.16it/s]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing equations: 0it [00:00, ?it/s]
Detecting bboxes: 0it [00:00, ?it/s]
100%|█████████▉| 771/773 [08:37<00:05,  2.73s/it]

converting D:\pdf_data\Évacuation des condensats dans le réseau d'évacuation du bâtiment.pdf to markdown ...


Recognizing Text: 100%|██████████| 1/1 [00:00<00:00,  2.17it/s]
Recognizing equations: 0it [00:00, ?it/s]
Detecting bboxes: 0it [00:00, ?it/s]
100%|█████████▉| 772/773 [08:38<00:02,  2.20s/it]

converting D:\pdf_data\Évacuation des condensats.pdf to markdown ...


Recognizing Text: 100%|██████████| 1/1 [00:00<00:00,  2.95it/s]
Recognizing equations: 0it [00:00, ?it/s]
Detecting bboxes: 0it [00:00, ?it/s]
100%|██████████| 773/773 [08:39<00:00,  1.49it/s]
